<h2 align="center">Loading Times.</h2>  
By: Jordi Tarragó i Vidal  

In [1]:
import pandas as pd
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text

In [20]:
from sqlalchemy import event

In [2]:
from dgxm import precio_horario

In [3]:
df_2018 = precio_horario('PrecBolsNaci', '2018-01-01', '2018-12-31')

In [4]:
df_2018.describe()

,PrecBolsNaci
count,8760.000000
mean,116.248822
std,49.623942
min,61.458540
25%,74.099580
50%,102.662830
75%,142.933030
max,555.119010


In [5]:
df_2018.isna().sum().sum()

0

In [6]:
df_2018.head(2)

,Fecha,PrecBolsNaci
0,2018-01-01 00:00:00,123.16729
1,2018-01-01 01:00:00,123.16729


In [7]:
df_2018.to_csv('Precios_Bolsa_2018.csv', index=False, encoding='utf-8')

In [8]:
with open('db_string', 'r') as cadena:
    engine = create_engine(cadena.readline())

In [14]:
with engine.connect() as conn:
    sql = text('DESCRIBE PrecioBolsa')
    results = conn.execute(sql)

In [15]:
for record in results:
    print("\n", record)


 ('Fecha', 'datetime', 'NO', 'PRI', None, '')

 ('PrecBolsNaci', 'decimal(12,5)', 'YES', '', None, '')


In [16]:
with engine.connect() as conn:
    sql = text('TRUNCATE PrecioBolsa')
    results = conn.execute(sql)

In [17]:
%%time

df_2018.to_sql('PrecioBolsa', engine, index=False, if_exists="append")

CPU times: user 56.8 ms, sys: 0 ns, total: 56.8 ms
Wall time: 2.09 s


8760

In [18]:
with engine.connect() as conn:
    sql = text('TRUNCATE PrecioBolsa')
    results = conn.execute(sql)

In [21]:
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(conn,
cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True

In [22]:
with engine.connect() as conn:
    sql = text('SELECT * FROM PrecioBolsa LIMIT 5')
    results = conn.execute(sql)
    print(results.all())

[]


In [23]:
%%time

df_2018.to_sql('PrecioBolsa', engine, index=False, if_exists="append")

CPU times: user 56.3 ms, sys: 0 ns, total: 56.3 ms
Wall time: 2.41 s


8760

In [24]:
with engine.connect() as conn:
    sql = text('SELECT * FROM PrecioBolsa LIMIT 5')
    results = conn.execute(sql)
    print(results.all())

[(datetime.datetime(2018, 1, 1, 0, 0), Decimal('123.16729')), (datetime.datetime(2018, 1, 1, 1, 0), Decimal('123.16729')), (datetime.datetime(2018, 1, 1, 2, 0), Decimal('123.16729')), (datetime.datetime(2018, 1, 1, 3, 0), Decimal('123.16729')), (datetime.datetime(2018, 1, 1, 4, 0), Decimal('123.16729'))]


In [25]:
engine.dispose()